In [ ]:
# Import python packages
from snowflake.core import Root

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Set up your role and warehouse
session.use_role("FRED_ROLE")  # Using FRED_ROLE as specified
session.use_warehouse("FRED_WH")

# Define your database and schema
database_name = "FRED_DB"
schema_name = "{{env}}_SCHEMA"  # This will be replaced by Jinja templating
env = "{{env}}"  # This will be replaced by Jinja templating

session.use_schema(f"{database_name}.{schema_name}")

# Import DAG-related modules
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask
from datetime import timedelta

# Create the tasks using the DAG API
warehouse_name = "FRED_WH"
dag_name = f"{env}_FRED_CURRENCY_PIPELINE"

api_root = Root(session)
schema = api_root.databases[database_name].schemas[schema_name]
dag_op = DAGOperation(schema)

# Define the DAG
with DAG(dag_name, schedule=timedelta(days=1), warehouse=warehouse_name) as dag:
    # Define tasks for each step in the pipeline
    # Each task executes a specific notebook for that stage of the pipeline
    
    # 1. Load Raw Data Task - Executes the load_raw_data notebook
    load_raw_data_task = DAGTask(
        "LOAD_RAW_DATA_TASK", 
        definition=f'''EXECUTE NOTEBOOK "{database_name}"."{env}_SCHEMA"."{env}_load_raw_data"()''', 
        warehouse=warehouse_name
    )
    
    # 2. Harmonize Data Task - Executes the harmonize_data notebook
    harmonize_data_task = DAGTask(
        "HARMONIZE_DATA_TASK", 
        definition=f'''EXECUTE NOTEBOOK "{database_name}"."{env}_SCHEMA"."{env}_harmonize_data"()''', 
        warehouse=warehouse_name
    )
    
    # 3. Analytics Task - Executes the analytics notebook
    analytics_task = DAGTask(
        "ANALYTICS_TASK", 
        definition=f'''EXECUTE NOTEBOOK "{database_name}"."{env}_SCHEMA"."{env}_analytics"()''', 
        warehouse=warehouse_name
    )

    # Define the dependencies between the tasks
    load_raw_data_task >> harmonize_data_task >> analytics_task

# Create the DAG in Snowflake
dag_op.deploy(dag, mode="orreplace")

# List all DAGs with similar names
dagiter = dag_op.iter_dags(like=f'{env}_FRED_CURRENCY_PIPELINE%')
for dag_name in dagiter:
    print(dag_name)

# Uncomment to run the DAG immediately
# dag_op.run(dag)